In [1]:
# Importing packages
import pandas as pd
from cassandra.cluster import Cluster

In [2]:
#connect to cluster

clstr = Cluster()
session = clstr.connect()

In [4]:
#drop keyspace bd23

session.execute("DROP KEYSPACE IF EXISTS bd23")

In [5]:
#describe all keyspaces that exist
rows = session.execute("desc keyspaces")
for row in rows:
    print(f"{row[0]}")

system
system_auth
system_distributed
system_schema
system_traces
system_views
system_virtual_schema


In [6]:
#create keyspace bd23

session.execute("""
    CREATE KEYSPACE IF NOT EXISTS bd23 WITH REPLICATION = {
        'class':'SimpleStrategy', 
        'replication_factor':3
    }

""")

In [7]:
#check that keyspace bd23 is created by describing all keyspaces

rows = session.execute("desc keyspaces")
for row in rows:
    print(f"{row[0]}")

bd23
system
system_auth
system_distributed
system_schema
system_traces
system_views
system_virtual_schema


In [8]:
# create a table to store inventory info

session.execute("""
CREATE TABLE IF NOT EXISTS bd23.inventoryData ( 
    sku INT, 
    product_name TEXT, 
    description TEXT, 
    warehouse_num INT,
    PRIMARY KEY(warehouse_num, sku, product_name)
);
""")

In [9]:
# load the data from a csv file

df = pd.read_csv('raghavData.csv')
df

,sku,product_name,description,warehouse_num
0,100000,'Wireless Keyboard','This wireless keyboard is great',123
1,200000,'Wireless Mouse','This wireless mouse has great range',123
2,300000,'Roku TV','4K 120Hz screen and internet connectivity',227
3,400000,'HDMI Cable','Braided nylon super fast HDMI Cable',301
4,500000,'Airpods','Roam tangle and cord-free with these airpods',747
5,600000,'Tapestry','Night sky themed Boho Chic Tapestry',404


In [10]:
# verify data 

for index, row in df.iterrows():
    print(f"sku = {row.sku}, product_name ={row.product_name}, description = {row.description}, warehouse_num = {row.warehouse_num}")

sku = 100000, product_name ='Wireless Keyboard', description = 'This wireless keyboard is great', warehouse_num = 123
sku = 200000, product_name ='Wireless Mouse', description = 'This wireless mouse has great range', warehouse_num = 123
sku = 300000, product_name ='Roku TV', description = '4K 120Hz screen and internet connectivity', warehouse_num = 227
sku = 400000, product_name ='HDMI Cable', description = 'Braided nylon super fast HDMI Cable', warehouse_num = 301
sku = 500000, product_name ='Airpods', description = 'Roam tangle and cord-free with these airpods', warehouse_num = 747
sku = 600000, product_name ='Tapestry', description = 'Night sky themed Boho Chic Tapestry', warehouse_num = 404


In [11]:
#view and insert data 

for index, row in df.iterrows():
    print(f"""
        INSERT INTO bd23.inventoryData (sku, product_name, description, warehouse_num)     
        VALUES ({row.sku}, {row.product_name}, {row.description}, {row.warehouse_num});
        """
    )
    session.execute(f"""
        INSERT INTO bd23.inventoryData (sku, product_name, description, warehouse_num)     
        VALUES ({row.sku}, {row.product_name}, {row.description}, {row.warehouse_num});
        """
    )


        INSERT INTO bd23.inventoryData (sku, product_name, description, warehouse_num)     
        VALUES (100000, 'Wireless Keyboard', 'This wireless keyboard is great', 123);
        

        INSERT INTO bd23.inventoryData (sku, product_name, description, warehouse_num)     
        VALUES (200000, 'Wireless Mouse', 'This wireless mouse has great range', 123);
        

        INSERT INTO bd23.inventoryData (sku, product_name, description, warehouse_num)     
        VALUES (300000, 'Roku TV', '4K 120Hz screen and internet connectivity', 227);
        

        INSERT INTO bd23.inventoryData (sku, product_name, description, warehouse_num)     
        VALUES (400000, 'HDMI Cable', 'Braided nylon super fast HDMI Cable', 301);
        

        INSERT INTO bd23.inventoryData (sku, product_name, description, warehouse_num)     
        VALUES (500000, 'Airpods', 'Roam tangle and cord-free with these airpods', 747);
        

        INSERT INTO bd23.inventoryData (sku, product_name,

In [12]:
# query all products at warehouse 123

rows = session.execute("select (sku, product_name, description, warehouse_num) from bd23.inventoryData where warehouse_num = 123")
for row in rows:
    print(f"{row[0][0]}, {row[0][1]}, {row[0][2]}, {row[0][3]}")

100000, Wireless Keyboard, This wireless keyboard is great, 123
200000, Wireless Mouse, This wireless mouse has great range, 123
